In [109]:
import numpy as np
from tqdm import tqdm
import netsquid as ns
import netsquid.qubits.qubitapi as qapi
from netsquid.components.qsource import SourceStatus
from netsquid.util import DataCollector
from netsquid_ae.datacollectors import ChainStateCollector
from netsquid_ae.protocol_event_types import *
from netsquid_ae.ae_chain_setup import create_repeater_link
from netsquid.qubits.ketstates import b00, b01, b10, b11, BellIndex

In [264]:
sim_params = {
            # SIMULATION
            "multi_photon": False,          # bool: whether to use multi-photon up to n=3 (WARNING: very slow)
            "num_repeaters": 0,             # number of repeaters
            "encoding": "time_bin",         # encoding of the entangled photons
            # end of simulation             Note: only use one and set others to -1
            "num_attempts": 1,             # number of clicks after which the clock stops (thus stopping the protocols)
            "num_attempts_proto": -1,       # number of clicks after which the emission protocols stop
            "num_successes": -1,            # number of successes after which to stop the simulation
            # magic
            "magic": None,                  # whether we want to use magic (should be "analytical", "sampled" or None)
            "state_files_directory": "./",
            # clock                         Note: only used if magic = None, otherwise cycle time is used as time_step
            "time_step": 1e6,               # time step for the clock [ns]
            "multiple_link_successes": False,  # whether elementary links can have multiple successful modes

            # COMPONENTS
            # channel
            "length": 0,                    # total distance between end node/ total channel length [km]
            "channel_length_l": -1,         # channel length left of the detectors [km]
            "channel_length_r": -1,         # channel length right of the detectors [km]
            "coupling_loss_fibre": 0.,      # initial loss on channel to midpoint detectors
            "attenuation_l": 0.25,          # channel attenuation left of the detectors [dB/km]
            "attenuation_r": 0.25,          # channel attenuation right of the detectors [dB/km]
            "fibre_phase_stdv_l": .0,
            "fibre_phase_stdv_r": .0,
            # source
            "source_frequency": 20e6,                 # frequency of the photon pair source [Hz]
            "num_multiplexing_modes": 1,
            "mean_photon_number": None,               # mean photon pair number (only used if multi_photon=True)
            "emission_probabilities": [0., 1., 0., 0.],  # emission probs for photon pair sources
            # midpoint detector
            "det_dark_count_prob": 0,   # probability of dark count per detection
            "det_efficiency": 1,       # detector efficiency
            "det_visibility": 1.,       # photon indistinguishability (must be 1 for multi_photon=True)
            "det_num_resolving": True,  # using number or non_number resolving detectors
            # swap & end node detector
            "swap_det_dark_count_prob": 0,   # probability of dark count per detection
            "swap_det_efficiency": 0.5,       # detector efficiency
            "swap_det_visibility": 1.,       # photon indistinguishability (must be 1 for multi_photon=True)
            "swap_det_num_resolving": True,  # using number or non_number resolving detectors
            # memory
            "memory_coherence_time": 1e6,               # coherence time of the quantum memory [ns]
            "max_memory_efficiency": 1.,                # maximum efficiency of the quantum memory
            "memory_time_dependence": "exponential",    # time-dependence of efficiency
        }

In [313]:
sim_params["length"] = 0.002
sim_params["channel_length_l"] = 0.001
sim_params["channel_length_r"] = 0.001

In [314]:
def run_simulation(sim_params):
    ns.sim_reset()
    ns.qubits.qformalism.set_qstate_formalism(ns.qubits.qformalism.QFormalism.KET)
    protocols, network, _ = create_elementary_link(**sim_params)
    for proto in protocols:
        proto.start()
    for node in network.nodes.values():
        node.subcomponents["Clock"].start()
    link_state_collector = DataCollector(ChainStateCollector(nodes_1=list(network.nodes.values())))
    link_state_collector.collect_on([(protocols[0], EVTYPE_SUCCESS), 
                                     (protocols[1], EVTYPE_SUCCESS),
                                    ], "AND")
    ns.sim_run()
    return link_state_collector.dataframe

In [317]:
count = 0
for i in tqdm(range(10000)):
    df = run_simulation(sim_params)
    count += len(df)
count

100%|█████████████████████████████████████████████████████████| 10000/10000 [00:51<00:00, 195.85it/s]


5048

In [303]:
q1, q2 = qapi.create_qubits(num_qubits=2)

In [305]:
for index, row in df.iterrows():
    q1, q2 = qapi.create_qubits(num_qubits=2)
    print(q1, q2)
    qapi.assign_qstate([q1, q2], row["state"])
    if row["midpoint_outcome_0"] == BellIndex.B01:
        print(np.isclose(qapi.fidelity([q1, q2], b01), 1))
    else:
        print(np.isclose(qapi.fidelity([q1, q2], b11), 1))

Qubit('QS#85784-0') Qubit('QS#85784-1')
True


In [318]:
df.head()

,time_stamp,entity_name,state,number_of_rounds,round,midpoint_outcome_0
0,10.0,EmissionProtocol('EmissionProtocol'),"[[0j, 0j, 0j, 0j], [0j, (0.49999999999999994+0...",1.0,1.0,2.0


In [325]:
for out in df["midpoint_outcome_0"]:
    if out == BellIndex.PSI_PLUS:
        print(BellIndex.PSI_PLUS)
    elif out == BellIndex.PSI_MINUS:
        print(BellIndex.PSI_MINUS)

BellIndex.B11
